In [0]:
order_items = spark.read.json('/public/retail_db_json/order_items')

In [0]:
order_items_grouped = order_items.groupBy('order_item_order_id')

In [0]:
order_items_grouped. \
    sum('order_item_quantity','order_item_subtotal'). \
    show()

+-------------------+------------------------+------------------------+
|order_item_order_id|sum(order_item_quantity)|sum(order_item_subtotal)|
+-------------------+------------------------+------------------------+
|                 29|                       9|                 1109.85|
|                474|                      13|       774.8199999999999|
|                964|                      11|       739.8800000000001|
|               1677|                      14|       649.9200000000001|
|               1806|                       8|                  789.94|
|               1950|                      12|      1015.8700000000001|
|               2214|                       5|                  449.96|
|               2250|                      10|                  889.94|
|               2453|                       7|       999.9300000000001|
|               2509|                       4|                  889.94|
|               2529|                       1|                  

In [0]:
type(order_items_grouped)

pyspark.sql.group.GroupedData

In [0]:
help(order_items_grouped.agg)

Help on method agg in module pyspark.sql.group:

agg(*exprs: Union[pyspark.sql.column.Column, Dict[str, str]]) -> pyspark.sql.dataframe.DataFrame method of pyspark.sql.group.GroupedData instance
    Compute aggregates and returns the result as a :class:`DataFrame`.
    
    The available aggregate functions can be:
    
    1. built-in aggregation functions, such as `avg`, `max`, `min`, `sum`, `count`
    
    2. group aggregate pandas UDFs, created with :func:`pyspark.sql.functions.pandas_udf`
    
       .. note:: There is no partial aggregation with group aggregate UDFs, i.e.,
           a full shuffle is required. Also, all the data of a group will be loaded into
           memory, so the user should be aware of the potential OOM risk if data is skewed
           and certain groups are too large to fit in memory.
    
       .. seealso:: :func:`pyspark.sql.functions.pandas_udf`
    
    If ``exprs`` is a single :class:`dict` mapping from string to string, then the key
    is the co

In [0]:
from pyspark.sql.functions import *

In [0]:
order_items_grouped. \
    agg(sum('order_item_quantity'), sum('order_item_subtotal')). \
    printSchema()

root
 |-- order_item_order_id: long (nullable = true)
 |-- sum(order_item_quantity): long (nullable = true)
 |-- sum(order_item_subtotal): double (nullable = true)



In [0]:
order_items_grouped. \
    agg(sum('order_item_quantity'), sum('order_item_subtotal')). \
    show()

+-------------------+------------------------+------------------------+
|order_item_order_id|sum(order_item_quantity)|sum(order_item_subtotal)|
+-------------------+------------------------+------------------------+
|                 29|                       9|                 1109.85|
|                474|                      13|       774.8199999999999|
|                964|                      11|       739.8800000000001|
|               1677|                      14|       649.9200000000001|
|               1806|                       8|                  789.94|
|               1950|                      12|      1015.8700000000001|
|               2214|                       5|                  449.96|
|               2250|                      10|                  889.94|
|               2453|                       7|       999.9300000000001|
|               2509|                       4|                  889.94|
|               2529|                       1|                  

In [0]:
order_items_grouped. \
    agg(sum('order_item_quantity').alias('order_quantity'), round(sum('order_item_subtotal'),2).alias('order_revenue')). \
    show()

+-------------------+--------------+-------------+
|order_item_order_id|order_quantity|order_revenue|
+-------------------+--------------+-------------+
|                 29|             9|      1109.85|
|                474|            13|       774.82|
|                964|            11|       739.88|
|               1677|            14|       649.92|
|               1806|             8|       789.94|
|               1950|            12|      1015.87|
|               2214|             5|       449.96|
|               2250|            10|       889.94|
|               2453|             7|       999.93|
|               2509|             4|       889.94|
|               2529|             1|        59.99|
|               2927|             8|       999.91|
|               3091|             5|       469.93|
|               3764|             2|        95.98|
|               4590|            11|       949.83|
|               4894|             4|       899.94|
|               5385|          

In [0]:
order_items_grouped. \
    agg({'order_item_quantity': 'sum', 'order_item_subtotal': 'sum' }). \
    printSchema()

root
 |-- order_item_order_id: long (nullable = true)
 |-- sum(order_item_subtotal): double (nullable = true)
 |-- sum(order_item_quantity): long (nullable = true)



In [0]:
order_items_grouped. \
    agg({'order_item_quantity': 'sum', 'order_item_subtotal': 'sum' }). \
    toDF('order_item_order_id','order_quantity','order_revenue'). \
    withColumn('order_revenue', round('order_revenue',2)). \
    printSchema()

root
 |-- order_item_order_id: long (nullable = true)
 |-- order_quantity: double (nullable = true)
 |-- order_revenue: long (nullable = true)



In [0]:
order_items_grouped. \
    agg({'order_item_quantity': 'sum', 'order_item_subtotal': 'sum' }). \
    toDF('order_item_order_id','order_quantity','order_revenue'). \
    withColumn('order_revenue', round('order_revenue',2)). \
    show()

+-------------------+------------------+-------------+
|order_item_order_id|    order_quantity|order_revenue|
+-------------------+------------------+-------------+
|                 29|           1109.85|            9|
|                474| 774.8199999999999|           13|
|                964| 739.8800000000001|           11|
|               1677| 649.9200000000001|           14|
|               1806|            789.94|            8|
|               1950|1015.8700000000001|           12|
|               2214|            449.96|            5|
|               2250|            889.94|           10|
|               2453| 999.9300000000001|            7|
|               2509|            889.94|            4|
|               2529|             59.99|            1|
|               2927| 999.9100000000001|            8|
|               3091|469.93000000000006|            5|
|               3764|             95.98|            2|
|               4590|            949.83|           11|
|         

In [0]:
# using dict has limitation. cannot specify same key twice for different aggregations. Even if we specify it will pick only one of them
order_items_grouped. \
    agg({'order_item_quantity': 'sum', 'order_item_quantity': 'min' }). \
    show()

+-------------------+------------------------+
|order_item_order_id|min(order_item_quantity)|
+-------------------+------------------------+
|                 29|                       1|
|                474|                       1|
|                964|                       1|
|               1677|                       1|
|               1806|                       1|
|               1950|                       1|
|               2214|                       1|
|               2250|                       1|
|               2453|                       1|
|               2509|                       1|
|               2529|                       1|
|               2927|                       1|
|               3091|                       1|
|               3764|                       2|
|               4590|                       1|
|               4894|                       1|
|               5385|                       1|
|               5409|                       1|
|            

In [0]:
order_items_grouped. \
    agg(sum('order_item_quantity').alias('order_quantity'), 
        min('order_item_quantity').alias('min_order_quantity'),
        round(sum('order_item_subtotal'),2).alias('order_revenue'),
        min('order_item_subtotal').alias('min_order_item_subtotal')
        ). \
    show()

+-------------------+--------------+------------------+-------------+-----------------------+
|order_item_order_id|order_quantity|min_order_quantity|order_revenue|min_order_item_subtotal|
+-------------------+--------------+------------------+-------------+-----------------------+
|                 29|             9|                 1|      1109.85|                 129.99|
|                474|            13|                 1|       774.82|                  24.99|
|                964|            11|                 1|       739.88|                 129.99|
|               1677|            14|                 1|       649.92|                   50.0|
|               1806|             8|                 1|       789.94|                  150.0|
|               1950|            12|                 1|      1015.87|                  87.96|
|               2214|             5|                 1|       449.96|                   50.0|
|               2250|            10|                 1|     